In [1]:
import pandas as pd

In [2]:
ted_data = pd.read_feather('ted_data')

In [5]:
ted_data.head()

,views,comments,duration,film_date,languages,num_speaker,published_date,Funny_ratings,Beautiful_ratings,Ingenious_ratings,...,is_ TED Fellows_tags_cleaned,is_ feminism_tags_cleaned,is_ TEDx_tags_cleaned,is_Gender equality_tags_cleaned,is_ journalism_tags_cleaned,is_ medical research_tags_cleaned,is_ disability_tags_cleaned,is_ security_tags_cleaned,is_ molecular biology_tags_cleaned,is_ algorithm_tags_cleaned
0,47227110,4553,1164,1140825600,60,1,1151367060,19645.0,4573.0,6073.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3200520,265,977,1140825600,43,1,1151367060,544.0,58.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1636292,124,1286,1140739200,26,1,1151367060,964.0,60.0,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1697550,200,1116,1140912000,35,1,1151367060,59.0,291.0,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12005869,593,1190,1140566400,48,1,1151440680,1390.0,942.0,3202.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y = ted_data.views
X = ted_data.drop(columns=['views'])

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [10]:
ted_data.isnull().values.any()

False

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=35)

In [12]:
from sklearn.feature_selection import RFECV
model = LinearRegression(n_jobs = -1)
rfecv = RFECV(model, cv = 5)
X_train_rfecv = rfecv.fit(X_train,y_train)

In [13]:
grid_scores = X_train_rfecv.grid_scores_
x_value_range = list(range(0, 287))[::-1]

In [14]:
from graph import trace_values, plot
score_trace = trace_values(x_value_range, grid_scores)
plot([score_trace])

In [15]:
grid_scores

array([-2.12675812e-03, -4.81135018e-03, -3.56047521e-03,  2.32841121e-02,
        2.44330360e-02,  2.30067855e-02,  2.00210819e-02,  9.27936188e-03,
        1.03730594e-02,  1.72367393e-02,  1.26780051e-02,  1.48517503e-02,
        1.12458252e-02,  6.13300597e-03,  1.76761049e-03,  9.12619832e-03,
        1.05282179e-02,  7.02636630e-03,  5.31321969e-03,  2.24584229e-03,
        8.47845612e-04,  1.52272365e-03,  2.73616455e-04, -6.28153213e-03,
       -5.11003546e-03, -3.43432242e-03, -8.20526888e-04,  2.10101612e-03,
        3.92961692e-03,  5.04079471e-03,  6.41782292e-03,  4.29313215e-03,
        2.42331334e-03,  2.12242568e-03,  4.40178271e-03,  3.03805688e-03,
        1.33278147e-03,  1.00530906e-03, -7.34844797e-04, -7.72966562e-04,
       -3.85396357e-04,  1.70699096e-03,  1.59565757e-03,  1.01283991e-03,
        4.85582966e-04, -5.97299216e-04,  4.87558895e-05,  2.03394833e-03,
        1.75142445e-03,  2.40212521e-04,  7.24369908e-05, -1.99878571e-03,
       -7.16384598e-04, -

In [16]:
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.33, random_state=35)

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import numpy as np

selectors = []

for idx in list(range(1, 288, 1))[::-1]:
    # 1. create an estimator
    estimator = LinearRegression(n_jobs=-1)
    # 2. perform rfe with the idx number of features
    rfe = RFE(estimator, idx, step=1)
    # 3. reduce the dataset to the number of features
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_validate_rfe = rfe.transform(X_validate)
    # 4. compute the accuracy score and append to the list
    estimator.fit(X_train_rfe,y_train)
    score = estimator.score(X_validate_rfe,y_validate)
    selectors.append(np.array([rfe, score, idx]))

In [18]:
wrapped_selectors = np.array(selectors)

In [19]:
y_values = wrapped_selectors[:, 1]
x_values = wrapped_selectors[:, -1]

In [20]:
from graph import trace_values, plot
trace = trace_values(x_values, y_values)
plot([trace])